In [112]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSPp3S5k4BfXDUeaz6wY9EzxGRk25RbAzVsRdUeKX8xHc6dwk5O")

# Airbnb in Lisbon

# Import Packages

In [34]:
import pandas as pd
import pandas_profiling

%matplotlib inline

## Let's get some data

In [72]:
#data from insideairbnb.com

reviews = '/Users/nonoumasy/Google Drive/Colab Notebooks/airbnb_lisbon/reviews.csv'
listings = '/Users/nonoumasy/Google Drive/Colab Notebooks/airbnb_lisbon/listings.csv'
#cal = '/Users/nonoumasy/Google Drive/Colab Notebooks/airbnb_lisbon/calendar.csv'

reviews = pd.read_csv(reviews, nrows=100)
listings = pd.read_csv(listings, nrows=100)
reviews = reviews.iloc[:,[0,2,3,4,5]]

#merge both dataframes
df = pd.concat([listings, reviews], axis=1)

# Preprocess Data

In [21]:
#quick check to see which neighborhood to use. Looks like neighborhood_cleansed is best
df[['neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed']].head()

,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed
0,Belém,Belm,Lisboa
1,Alfama,Santa Maria Maior,Lisboa
2,NaN,Aveiras de Cima,Azambuja
3,São João de Deus,Areeiro,Lisboa
4,Alfama,Santa Maria Maior,Lisboa


In [73]:
#keep only relevant variables
df = df[['id',
         'name',
         'summary',
         'space',
         'description',
         'neighborhood_overview',
         'notes',
         'transit',
         'neighbourhood_cleansed',
         'latitude',
         'longitude',
         'property_type',
         'room_type',
         'square_feet',
         'price', 
         'weekly_price',                                   
         'monthly_price', 
         'square_feet',
         'availability_30',
         'review_scores_rating',
         'review_scores_location',
         'comments']]


#remove symbols from price and cast it to float
df.price = [x.strip('$') for x in df.price]
df['price'] = df['price'].str.replace(',','')
df['price'] = pd.to_numeric(df.price)

In [74]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 22 columns):
id                        100 non-null int64
name                      100 non-null object
summary                   66 non-null object
space                     99 non-null object
description               100 non-null object
neighborhood_overview     54 non-null object
notes                     49 non-null object
transit                   52 non-null object
neighbourhood_cleansed    100 non-null object
latitude                  100 non-null float64
longitude                 100 non-null float64
property_type             100 non-null object
room_type                 100 non-null object
square_feet               34 non-null float64
price                     100 non-null object
weekly_price              47 non-null object
monthly_price             44 non-null object
square_feet               34 non-null float64
availability_30           100 non-null int64
review_scores_rating      82 

# Summary Statistics

In [93]:
#summary stats
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,100.0,111704.820000,44378.613700,6499.00000,83678.750000,125841.50000,147043.500000,174529.00000
latitude,100.0,38.745486,0.082077,38.68647,38.711647,38.71525,38.738235,39.14654
longitude,100.0,-9.192959,0.111208,-9.47342,-9.175675,-9.14830,-9.132560,-8.91503
square_feet,34.0,634.970588,1180.131926,0.00000,41.250000,377.00000,834.000000,6889.00000
price,100.0,92.420000,116.773819,17.00000,47.500000,65.00000,83.500000,1000.00000
square_feet,34.0,634.970588,1180.131926,0.00000,41.250000,377.00000,834.000000,6889.00000
availability_30,100.0,11.740000,11.143345,0.00000,3.000000,7.00000,21.250000,30.00000
review_scores_rating,82.0,92.512195,6.992490,60.00000,90.000000,94.00000,97.000000,100.00000
review_scores_location,81.0,9.518519,0.672888,7.00000,9.000000,10.00000,10.000000,10.00000


In [94]:
#Number of listings in Lisbon
print('Number of Lisbon listings ', listings.shape[0])

#Number of reviews in Lisbon listings
print('Number of Reviews in Lisbon listings ', reviews.shape[0])

Number of Lisbon listings  100
Number of Reviews in Lisbon listings  100


In [97]:
#What is the average price per night in dollars?
df.price.mean()

92.42

# Neighborhoods - Let's get to know the area

In [105]:
#Number of Lisbon neighborhoods
df.neighbourhood_cleansed.nunique()

32

In [107]:
#Lisbon neighborhoods
df.neighbourhood_cleansed.value_counts()[:10]

Santa Maria Maior                                    21
Misericrdia                                          14
So Vicente                                           10
Estrela                                               8
Cascais e Estoril                                     6
Santo Antnio                                          5
Avenidas Novas                                        3
Alvalade                                              3
S.Maria, S.Miguel, S.Martinho, S.Pedro Penaferrim     3
Colares                                               3
Name: neighbourhood_cleansed, dtype: int64

# Listings

In [109]:
df.head()

,id,name,summary,space,description,neighborhood_overview,notes,transit,neighbourhood_cleansed,latitude,...,room_type,square_feet,price,weekly_price,monthly_price,square_feet,availability_30,review_scores_rating,review_scores_location,comments
0,6499,Belém 1 Bedroom Historical Apartment,"This apartment is all about Location, next to ...",Fully furnished apartment (35m2). Small inside...,"This apartment is all about Location, next to ...","To get to the city center, tram 15 stops in f...",Late check in's (after 20H) have an extra char...,The apartment is on the same street as the #15...,Belm,38.69675,...,Entire home/apt,377.0,36.0,$209.00,$579.00,377.0,5,98.0,10.0,"Ola Bruno,\r\n\r\nTive um mes Fantástico em se..."
1,25659,"Sunny, Alfama Sleeps 3 - Coeur d'Alfama",Heart of Alfama! Center of Lisbon. Sunny Char...,Charming sunny apartment in the heart of Alfam...,Heart of Alfama! Center of Lisbon. Sunny Char...,Alfama is the oldest and most picturesque quar...,Check in after 3 PM although we try to accommo...,The apartment is very convenient to all kinds ...,Santa Maria Maior,38.71167,...,Entire home/apt,NaN,60.0,$350.00,NaN,NaN,5,96.0,10.0,Encontramos o apartamento de Bruno exatamente ...
2,28066,ear the birds,NaN,traditional farm house in a quiet place where ...,traditional farm house in a quiet place where ...,NaN,NaN,NaN,Aveiras de Cima,39.14654,...,Private room,NaN,53.0,NaN,NaN,NaN,30,NaN,NaN,Estivemos em Lisboa por aproximadamente 03 (tr...
3,29138,Room for rent in shared house,NaN,"05 minutes from city center by subway, 10 minu...","05 minutes from city center by subway, 10 minu...",NaN,NaN,NaN,Areeiro,38.73634,...,Private room,NaN,71.0,NaN,NaN,NaN,30,NaN,NaN,Superbe quartier très proche du tram et du tra...
4,29248,Apartamento Alfama com vista para o rio!,NaN,"Located in the city center, and great location...","Located in the city center, and great location...",NaN,NaN,NaN,Santa Maria Maior,38.71272,...,Entire home/apt,NaN,60.0,$299.00,$966.00,NaN,4,91.0,10.0,"Très bel appartement, bien situé et à proximit..."
